In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import cv2
import re
# import os
import glob
# import re
# import numpy as np
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
import tifffile as tiff
# import os
# import numpy as np
# import glob
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
# import rasterio

import json

with open('config.json') as json_file:
    config = json.load(json_file)
    corruption = config['corruption_level']


def load_image(filepath):
    # Load the image using PIL
    image = np.array(Image.open(filepath).convert('L'))
    
    # Apply Otsu's thresholding
    t, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Convert to binary (0 and 1)
    binary_image = (binary_image > 0).astype(np.uint8)
    return binary_image

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred))

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)

# Paths to the folders containing the predictions and ground truths


In [2]:
# gt_folder = './GEE_Masks/GEE_resized/'
# pred_folder = './GEE_Output/Adversarial'

# # List all files in the prediction folder
# # pred_files = [f for f in os.listdir(pred_folder) if f.endswith('.tif')]

# # Initialize metrics



In [3]:
# # Lists to store metrics
# dice_scores = []
# iou_scores = []
# pixel_accuracies = []
# precisions = []
# recalls = []
# f1_scores = []
# specificities = []

# for i in range(0, 1263):  # Adjust the range based on the number of files
#     gt_filename = f'NDWI_Mask_{i}_resized.tif'
#     pred_filename = f'dense_Predicted_Image_{i}.tif'
    
#     gt_path = os.path.join(gt_folder, gt_filename)
#     pred_path = os.path.join(pred_folder, pred_filename)
    
#     # Load the prediction and ground truth images
#     pred_image = load_image(pred_path)
#     gt_image = load_image(gt_path)
    
#     # Flatten the images
#     pred_flat = pred_image.flatten()
#     gt_flat = gt_image.flatten()
    
#     # Calculate metrics
#     dice = dice_coefficient(gt_flat, pred_flat)
#     iou = jaccard_score(gt_flat, pred_flat, average='macro')
#     accuracy = pixel_accuracy(gt_flat, pred_flat)
#     precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
#     recall = recall_score(gt_flat, pred_flat, average='macro')
#     f1 = f1_score(gt_flat, pred_flat, average='macro')
#     spec = specificity(gt_flat, pred_flat)
    
#     # Store metrics
#     dice_scores.append(dice)
#     iou_scores.append(iou)
#     pixel_accuracies.append(accuracy)
#     precisions.append(precision)
#     recalls.append(recall)
#     f1_scores.append(f1)
#     specificities.append(spec)
    
#     # Print metrics for the current image
#     print(f"Metrics for {gt_filename} and {pred_filename}:")
#     print(f"  Dice Coefficient: {dice:.4f}")
#     print(f"  IoU: {iou:.4f}")
#     print(f"  Pixel Accuracy: {accuracy:.4f}")
#     print(f"  Precision: {precision:.4f}")
#     print(f"  Recall: {recall:.4f}")
#     print(f"  F1 Score: {f1:.4f}")
#     print(f"  Specificity: {spec:.4f}")
#     print()

# # Calculate and print average metrics
# average_dice = np.mean(dice_scores)
# average_iou = np.mean(iou_scores)
# average_accuracy = np.mean(pixel_accuracies)
# average_precision = np.mean(precisions)
# average_recall = np.mean(recalls)
# average_f1 = np.mean(f1_scores)
# average_specificity = np.mean(specificities)

# print("Average metrics:")
# print(f"  Average Dice Coefficient: {average_dice:.4f}")
# print(f"  Average IoU: {average_iou:.4f}")
# print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
# print(f"  Average Precision: {average_precision:.4f}")
# print(f"  Average Recall: {average_recall:.4f}")
# print(f"  Average F1 Score: {average_f1:.4f}")
# print(f"  Average Specificity: {average_specificity:.4f}")

# # Model's overall accuracy
# print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

In [4]:
# # Print average metrics
# print("Average Metrics:")
# print(f"  Average Dice Coefficient: {np.mean(dice_scores):.4f}")
# print(f"  Average IoU: {np.mean(iou_scores):.4f}")
# print(f"  Average Pixel Accuracy: {np.mean(pixel_accuracies):.4f}")
# print(f"  Average Precision: {np.mean(precisions):.4f}")
# print(f"  Average Recall: {np.mean(recalls):.4f}")
# print(f"  Average F1 Score: {np.mean(f1_scores):.4f}")
# print(f"  Average Specificity: {np.mean(specificities):.4f}")

In [5]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)


# Directories for ground truth and prediction masks
gt_folder = './GEE_Masks/GEE_resized/'


# For UNet
# pred_folder = f'./GEE_Output/Adversarial/New_20_Epoch_{corruption}_with_diff_kernels'
# pred_files = glob.glob(os.path.join(pred_folder, 'dense_*.tif'))

# For DeepLabV3+
pred_folder = f'./GEE_Output/Adversarial/New_20_Epoch_DeepLab_{corruption}_with_diff_kernels'
print(pred_folder)

# Initialize metric lists
dice_scores = []
iou_scores = []
pixel_accuracies = []
precisions = []
recalls = []
f1_scores = []
specificities = []

# Get all prediction files

pred_files = glob.glob(os.path.join(pred_folder, 'mobilenetv2_*.tif'))

for pred_file in pred_files:
    # Extract the number 'i' from the prediction filename
    # match = re.search(r'dense_(\d+).tif', os.path.basename(pred_file))
    match = re.search(r'mobilenetv2_(\d+).tif', os.path.basename(pred_file))
    if match:
        i = match.group(1)
        gt_filename = f'NDWI_Mask_{i}_resized.tif'
        gt_path = os.path.join(gt_folder, gt_filename)
        
        # Check if the ground truth file exists
        if os.path.exists(gt_path):
            # Load the prediction and ground truth images
            pred_image = load_image(pred_file)
            gt_image = load_image(gt_path)
            print(np.max(pred_image))
            print(np.min(pred_image))
            # Flatten the images
            pred_flat = pred_image.flatten()
            gt_flat = gt_image.flatten()
            
            # Calculate metrics
            dice = dice_coefficient(gt_flat, pred_flat)
            iou = jaccard_score(gt_flat, pred_flat, average='macro')
            accuracy = pixel_accuracy(gt_flat, pred_flat)
            precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
            recall = recall_score(gt_flat, pred_flat, average='macro')
            f1 = f1_score(gt_flat, pred_flat, average='macro')
            spec = specificity(gt_flat, pred_flat)
            
            # Store metrics
            dice_scores.append(dice)
            iou_scores.append(iou)
            pixel_accuracies.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)
            specificities.append(spec)
            
            # Print metrics for the current image
            print(f"Metrics for {gt_filename} and {pred_file}:")
            print(f"  Dice Coefficient: {dice:.4f}")
            print(f"  IoU: {iou:.4f}")
            print(f"  Pixel Accuracy: {accuracy:.4f}")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall: {recall:.4f}")
            print(f"  F1 Score: {f1:.4f}")
            print(f"  Specificity: {spec:.4f}")
            print()
        else:
            print(f"Ground truth file {gt_path} not found for prediction file {pred_file}.")

# Calculate and print average metrics
average_dice = np.mean(dice_scores)
average_iou = np.mean(iou_scores)
average_accuracy = np.mean(pixel_accuracies)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)
average_specificity = np.mean(specificities)

print("Average Metrics:")
print(f"  Dice Coefficient: {average_dice:.4f}")
print(f"  IoU: {average_iou:.4f}")
print(f"  Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Precision: {average_precision:.4f}")
print(f"  Recall: {average_recall:.4f}")
print(f"  F1 Score: {average_f1:.4f}")
print(f"  Specificity: {average_specificity:.4f}")

./GEE_Output/Adversarial/New_20_Epoch_DeepLab_0_with_diff_kernels
0
0
Metrics for NDWI_Mask_10_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_DeepLab_0_with_diff_kernels\mobilenetv2_10.tif:
  Dice Coefficient: 0.0000
  IoU: 0.3026
  Pixel Accuracy: 0.6052
  Precision: 0.3026
  Recall: 0.5000
  F1 Score: 0.3770
  Specificity: 1.0000

0
0
Metrics for NDWI_Mask_1008_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_DeepLab_0_with_diff_kernels\mobilenetv2_1008.tif:
  Dice Coefficient: 0.0000
  IoU: 0.4596
  Pixel Accuracy: 0.9192
  Precision: 0.4596
  Recall: 0.5000
  F1 Score: 0.4790
  Specificity: 1.0000

0
0
Metrics for NDWI_Mask_1010_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_DeepLab_0_with_diff_kernels\mobilenetv2_1010.tif:
  Dice Coefficient: 0.0000
  IoU: 0.3302
  Pixel Accuracy: 0.6605
  Precision: 0.3302
  Recall: 0.5000
  F1 Score: 0.3978
  Specificity: 1.0000

0
0
Metrics for NDWI_Mask_1012_resized.tif and ./GEE_Output/Adversarial/New_20_Epoch_DeepLab_0_wit

In [6]:
print("Average metrics:")
print(f"  Average Dice Coefficient: {average_dice:.4f}")
print(f"  Average IoU: {average_iou:.4f}")
print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Average Precision: {average_precision:.4f}")
print(f"  Average Recall: {average_recall:.4f}")
print(f"  Average F1 Score: {average_f1:.4f}")
print(f"  Average Specificity: {average_specificity:.4f}")

# Model's overall accuracy
print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

Average metrics:
  Average Dice Coefficient: 0.0864
  Average IoU: 0.4394
  Average Pixel Accuracy: 0.7997
  Average Precision: 0.4394
  Average Recall: 0.5395
  Average F1 Score: 0.4763
  Average Specificity: 1.0000
Model's Overall Accuracy: 0.7997
